In [340]:
# import necessary libraries
import pandas as pd
import os
import glob


In [341]:
data_path = os.getcwd() + '\\data_raw\\'

def make_dfs(path):
    # use glob to get all the csv files in the folder
    csv_files = glob.glob(os.path.join(path, '*.csv'))

    age_df_list = []
    income_df_list = []

    for filename in csv_files:
        l = filename.split('.')[0].split('\\')[-1].split('_')[1::]

        df = pd.read_csv(filename, skiprows=4)
        df['Metric'] = l[1]
        df['Year'] = l[2]

        if l[0] == 'age':
            age_df_list.append(df)
        elif l[0] == 'income':
            income_df_list.append(df)

    df_age = pd.concat(age_df_list, axis=0)
    df_income = pd.concat(income_df_list, axis=0)

    return [df_age, df_income]

dfs = make_dfs(data_path)

df_age = dfs[0]
df_income = dfs[1]

df_age

,Selected Geographies,0-9 Years Old,10-19 Years Old,20-29 Years Old,30-39 Years Old,40-49 Years Old,50-59 Years Old,60-69 Years Old,70-79 Years Old,80-89 Years Old,90-99 Years Old,Metric,Year
0,Los Angeles County (North/Unincorporated)--Cas...,5,5,4,4,4,3,3,2,2,4.0,np,2015
1,Los Angeles County (Northwest)--Santa Clarita ...,5,5,4,4,4,3,3,3,2,2.0,np,2015
2,Los Angeles County (North Central)--Lancaster ...,5,5,4,4,4,3,3,2,2,2.0,np,2015
3,Los Angeles County (North Central)--Palmdale C...,5,5,4,5,4,3,3,3,3,2.0,np,2015
4,Los Angeles County (North)--LA City (Northwest...,5,5,4,4,4,3,3,2,2,3.0,np,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,Riverside County (Northwest)--Riverside City (...,655,678,615,609,687,388,460,249,581,787.0,rntp,2019
11,Riverside County (West Central)--Corona City (...,399,301,396,325,357,163,294,265,522,1308.0,rntp,2019
12,Riverside County (West Central)--Corona (North...,668,680,616,846,705,340,488,336,74,466.0,rntp,2019
13,Riverside County (Northwest)--Jurupa Valley & ...,599,630,618,609,353,259,303,256,754,181.0,rntp,2019


In [342]:
df_age = df_age.melt(id_vars=['Selected Geographies', 'Metric', 'Year'], var_name='Average Age Range', value_name='Value')
df_age = pd.pivot_table(df_age, values = 'Value', index=['Selected Geographies', 'Year', 'Average Age Range'], columns = ['Metric']).reset_index()

df_income = df_income.melt(id_vars=['Selected Geographies', 'Metric', 'Year'], var_name='Average Household Income', value_name='Value')
df_income = pd.pivot_table(df_income, values = 'Value', index=['Selected Geographies', 'Year', 'Average Household Income'], columns = ['Metric']).reset_index()

df_age.rename(columns= {'np':'Ave People in Household', 'rntp': 'Ave Rent per Month'}, inplace=True)
df_income.rename(columns= {'np':'Ave People in Household', 'rntp': 'Ave Rent per Month'}, inplace=True)

df_age[['County', 'County Region']] = df_age['Selected Geographies'].str.split('County', 1, expand=True)
df_age[['County Region', 'Cities']] = df_age['County Region'].str.split('--', 1, expand=True)
df_age[['Cities', 'Other']] = df_age['Cities'].str.split(r"\bPUMA\b", 1, expand=True)
df_age['County'] = df_age['County'].str.strip()
df_age['County Region'] = df_age['County Region'].str.strip(' ()')
df_age['Cities'] = df_age['Cities'].str.strip().str.replace("'",'')
df_age.drop(columns=['Selected Geographies', 'Other'], inplace=True)

df_income[['County', 'County Region']] = df_income['Selected Geographies'].str.split('County', 1, expand=True)
df_income[['County Region', 'Cities']] = df_income['County Region'].str.split('--', 1, expand=True)
df_income[['Cities', 'Other']] = df_income['Cities'].str.split(r"\bPUMA\b", 1, expand=True)
df_income['County'] = df_income['County'].str.strip()
df_income['County Region'] = df_income['County Region'].str.strip(' ()')
df_income['Cities'] = df_income['Cities'].str.strip()
df_income.drop(columns=['Selected Geographies', 'Other'], inplace=True)


'Bell Gardens, Bell, Maywood, Cudahy & Commerce Cities'

In [343]:
df_age.to_csv('Processed_Data\\age_data.csv', index=False)
df_income.to_csv('Processed_Data\\income_data.csv', index=False)